In [1]:
!pip install faker


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [26]:
import pyspark
from faker import Faker
from pyspark.sql import SparkSession, DataFrame, functions as F, types as t
from pyspark.sql.window import Window
import random
from datetime import datetime, timedelta, date

spark = SparkSession \
    .builder \
    .master("local[1]") \
    .appName("pyspark-test01") \
    .config("spark.sql.warehouse.dir", "spark-warehouse") \
    .enableHiveSupport() \
    .getOrCreate()
print(spark.catalog.listDatabases())

fake = Faker()
Faker.seed(42)
random.seed(42)


24/04/01 11:12:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='file:/workspaces/devcontainer-universal/spark-warehouse')]


In [133]:
@F.udf(t.IntegerType())
def f_int(min:int, max:int) -> int:
    return fake.random_int(min=min, max=max)

@F.udf
def f_intUnique(digits:int) -> int:
    return fake.unique.random_number(digits=digits)

@F.udf
def f_choise(list:list, weights:list) -> str:
    return random.choices(list,weights=weights)[0]

@F.udf
def f_choise_active() -> str:
    return random.choices(['Active', 'Inactive'],weights=[0.8,0.2])[0]

@F.udf
def f_bool() -> bool:
    return fake.boolean()

@F.udf
def f_GUID() -> str:
    return fake.uuid4()

@F.udf
def f_hash() -> str:
    return fake.md5(raw_output=False)

@F.udf
def f_date(start:str='-3y', end:str='now') -> str:
    return str(fake.date_between(start_date=start, end_date=end))

@F.udf
def f_timestamp(start:str='-3y', end:str='now') -> str:
    return str(fake.date_time_between(start_date=start, end_date=end))


In [134]:
consumers = spark.read.table('consumer_dimension').select('Consumer_Key')
consumers.count()
'''
  Consumer_Loyalty_Key BIGINT,
  Consumer_Key BIGINT,
  Consumer_Loyalty_Program_Level_Key BIGINT,
  Consumer_Loyalty_Membership_Identifier STRING,
  Consumer_Loyalty_Record_Valid_From_Timestamp TIMESTAMP,
  Consumer_Loyalty_Record_Valid_To_Timestamp TIMESTAMP,
  Consumer_Loyalty_Current_Record_Indicator TINYINT,
  Consumer_Loyalty_Anniversary_Date DATE,
  Consumer_Loyalty_Level_Code STRING,
  Consumer_Loyalty_Level_Description STRING,
  Consumer_Loyalty_Status_Code STRING,
  Consumer_Loyalty_Enrollment_Channel_Code STRING,
  Consumer_Loyalty_Start_Timestamp TIMESTAMP,
  Consumer_Loyalty_Anniversary_End_Date DATE,
  Consumer_Loyalty_Available_Point_Number INT,
  Consumer_Loyalty_Pending_Point_Number INT,
  Consumer_Loyalty_Qualifying_Point_Number INT,
  Consumer_Loyalty_Available_Amount DECIMAL(14,2),
  Consumer_Loyalty_Pending_Amount DECIMAL(14,2),
  Consumer_Loyalty_Qualifying_Amount DECIMAL(14,2),
  Consumer_Loyalty_Last_Activity_Timestamp TIMESTAMP,
  Consumer_Loyalty_Active_Indicator TINYINT,
  Consumer_Loyalty_Record_Created_Timestamp TIMESTAMP,
  Consumer_Loyalty_Record_Modified_Timestamp TIMESTAMP
'''
tmp_df = (consumers
          .withColumn('Consumer_Loyalty_Key', f_intUnique(F.lit(6)))
          .withColumn('Consumer_Loyalty_Program_Level_Key', f_int(F.lit(1),F.lit(10000))) # BIGINT,
          .withColumn('Consumer_Loyalty_Membership_Identifier', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Record_Valid_From_Timestamp', f_timestamp()) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Record_Valid_To_Timestamp', f_timestamp()) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Current_Record_Indicator', f_bool()) # TINYINT,
          .withColumn('Consumer_Loyalty_Anniversary_Date', f_date()) # DATE,
          .withColumn('Consumer_Loyalty_Level_Code', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Level_Description', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Status_Code', f_choise_active()) # STRING,
          .withColumn('Consumer_Loyalty_Enrollment_Channel_Code', F.lit('STRING')) # STRING,
          .withColumn('Consumer_Loyalty_Start_Timestamp', f_timestamp()) # TIMESTAMP,
          .withColumn('Consumer_Loyalty_Anniversary_End_Date', f_date()) # DATE,
        #   .withColumn('Consumer_Loyalty_Available_Point_Number', f_()) # INT,
        #   .withColumn('Consumer_Loyalty_Pending_Point_Number', f_()) # INT,
        #   .withColumn('Consumer_Loyalty_Qualifying_Point_Number', f_()) # INT,
        #   .withColumn('Consumer_Loyalty_Available_Amount', f_()) # DECIMAL(14,2),
        #   .withColumn('Consumer_Loyalty_Pending_Amount', f_()) # DECIMAL(14,2),
        #   .withColumn('Consumer_Loyalty_Qualifying_Amount', f_()) # DECIMAL(14,2),
        #   .withColumn('Consumer_Loyalty_Last_Activity_Timestamp', f_()) # TIMESTAMP,
        #   .withColumn('Consumer_Loyalty_Active_Indicator', f_()) # TINYINT,
        #   .withColumn('Consumer_Loyalty_Record_Created_Timestamp', f_()) # TIMESTAMP,
        #   .withColumn('Consumer_Loyalty_Record_Modified_Timestamp', f_()) # TIMESTAMP
)
tmp_df.show()

+------------+--------------------+----------------------------------+--------------------------------------+--------------------------------------------+------------------------------------------+-----------------------------------------+---------------------------------+---------------------------+----------------------------------+----------------------------+----------------------------------------+--------------------------------+-------------------------------------+
|Consumer_Key|Consumer_Loyalty_Key|Consumer_Loyalty_Program_Level_Key|Consumer_Loyalty_Membership_Identifier|Consumer_Loyalty_Record_Valid_From_Timestamp|Consumer_Loyalty_Record_Valid_To_Timestamp|Consumer_Loyalty_Current_Record_Indicator|Consumer_Loyalty_Anniversary_Date|Consumer_Loyalty_Level_Code|Consumer_Loyalty_Level_Description|Consumer_Loyalty_Status_Code|Consumer_Loyalty_Enrollment_Channel_Code|Consumer_Loyalty_Start_Timestamp|Consumer_Loyalty_Anniversary_End_Date|
+------------+--------------------+---------

In [45]:
groupBy_columns = ['Division_Code_Key','Region_Code_Key','Affiliate_Code_Key','Market_Code_Key']

# Total Buyers, will join it to Consumer Dimention on Consumer Master Reg GUID
trans_head = spark.read.table('transaction_header').select('Source_Consumer_Key')
master_reg = spark.read.table('master_registry').select('Source_Consumer_Key', 'Consumer_Master_Registry_Global_Unique_Identifier')
total_buyers = (
    master_reg
    .join(trans_head, master_reg.Source_Consumer_Key == trans_head.Source_Consumer_Key, 'left')
)

In [ ]:
# Lifecycle
lifecycle = (
    spark.read.table('consumer_lifecycle')
    .select('Consumer_Key','Consumer_Lifecycle_Status_Code_Key')
    .where(F.col('Consumer_Lifecycle_Status_Current_Record_Indicator') == 1)
)

In [113]:
# Loyalty
# loyalty = spark.read.table('consumer_loyalty')
spark.sql(
    '''
    CREATE TABLE consumer_loyalty (
  Consumer_Loyalty_Key BIGINT NOT NULL,
  Consumer_Key BIGINT NOT NULL,
  Consumer_Loyalty_Program_Level_Key BIGINT,
  Consumer_Loyalty_Membership_Identifier STRING NOT NULL,
  Consumer_Loyalty_Record_Valid_From_Timestamp TIMESTAMP NOT NULL,
  Consumer_Loyalty_Record_Valid_To_Timestamp TIMESTAMP NOT NULL,
  Consumer_Loyalty_Current_Record_Indicator TINYINT,
  Consumer_Loyalty_Anniversary_Date DATE,
  Consumer_Loyalty_Level_Code STRING,
  Consumer_Loyalty_Level_Description STRING,
  Consumer_Loyalty_Status_Code STRING,
  Consumer_Loyalty_Enrollment_Channel_Code STRING,
  Consumer_Loyalty_Start_Timestamp TIMESTAMP,
  Consumer_Loyalty_Anniversary_End_Date DATE,
  Consumer_Loyalty_Available_Point_Number INT,
  Consumer_Loyalty_Pending_Point_Number INT,
  Consumer_Loyalty_Qualifying_Point_Number INT,
  Consumer_Loyalty_Available_Amount DECIMAL(14,2),
  Consumer_Loyalty_Pending_Amount DECIMAL(14,2),
  Consumer_Loyalty_Qualifying_Amount DECIMAL(14,2),
  Consumer_Loyalty_Last_Activity_Timestamp TIMESTAMP,
  Consumer_Loyalty_Active_Indicator TINYINT,
  Consumer_Loyalty_Record_Created_Timestamp TIMESTAMP NOT NULL,
  Consumer_Loyalty_Record_Modified_Timestamp TIMESTAMP NOT NULL)
    '''
)

24/04/01 17:22:13 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
24/04/01 17:22:13 WARN HiveMetaStore: Location: file:/workspaces/devcontainer-universal/spark-warehouse/consumer_loyalty specified for non-external table:consumer_loyalty


DataFrame[]

In [74]:
# Contactables
phone = spark.read.table('consumer_phone').select('Consumer_Key','Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Phone_Current_Record_Indicator') == 1)
email = spark.read.table('consumer_email').select('Consumer_Key','Consumer_Email_Opt_In_Opt_Out_Indicator').where(F.col('Consumer_Email_Current_Record_Indicator') == 1)
address = spark.read.table('consumer_address').select('Consumer_Key','Consumer_Address_Postal_Opt_In_Opt_Out_Indicator','Consumer_Address_Current_Record_Indicator').where(F.col('Consumer_Address_Current_Record_Indicator')==1)
consumers = spark.read.table('consumer_dimension').select('Consumer_Key', *groupBy_columns)
contactable = (consumers
               .join(phone, consumers.Consumer_Key == phone.Consumer_Key, 'left').drop(phone.Consumer_Key)
               .join(email, consumers.Consumer_Key == email.Consumer_Key, 'left').drop(email.Consumer_Key)
               .join(address, consumers.Consumer_Key == address.Consumer_Key, 'left').drop(address.Consumer_Key)
)
contactable.show()


+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+------------------------------------------------+-----------------------------------------+
|Consumer_Key|Division_Code_Key|Region_Code_Key|Affiliate_Code_Key|Market_Code_Key|Consumer_Phone_SMS_Opt_In_Opt_Out_Indicator|Consumer_Email_Opt_In_Opt_Out_Indicator|Consumer_Address_Postal_Opt_In_Opt_Out_Indicator|Consumer_Address_Current_Record_Indicator|
+------------+-----------------+---------------+------------------+---------------+-------------------------------------------+---------------------------------------+------------------------------------------------+-----------------------------------------+
|        1701|                1|             10|            886963|              1|                                          0|                                   NULL|                                            NULL|       